In [ ]:
% reset -f

In [ ]:
import codecs
import time

import numpy as np
import pandas as pd
import papermill as pm
import pickle

In [ ]:
# tüm optimizer lar 'optimizers':
# hiperparameter list
hyper_parameters = {'epoch': 60, 'batch_size': 120, 'layer_size': [1, 2, 4], 'unit_size': [8, 16, 32],
                    'activations': ['relu', 'sigmoid', 'softmax', 'softplus', 'softsign', 'tanh', 'selu', 'elu'],
                    'loss_functions': ['mse', 'mape', 'mae'], 'models': ['GRU', 'LSTM', 'SimpleRNN'],
                    'optimizers': ["SGD", "RMSprop", "Adam", "Adadelta", "Adagrad", "Adamax", "Nadam", "Ftrl"]}

learning_rate_ranges = [0.0001, 0.09]
dropout_rate_ranges = [0.1, 0.5]
ga_parameters = {'max_num_iteration': 50,
                 'population_size': 10,
                 'crossover_chromosome_size': 6,
                 'mutation_chromosome_size': 3,
                 'crossover_rate': 0.6,
                 'mutation_rate': 0.3, 'crossover_point_size': 1, 'mutation_point_size': 3,
                 'prob_chr': [0.3, 0.2, 0.15, 0.12, 0.10, 0.07, 0.03, 0.02, 0.006, 0.004]}

In [ ]:
class Chromosome:
    def __init__(self, chr_id=0):
        """

        :param chr_id: Kromozomun çağrılması için değer
        """
        self.chr_id = chr_id
        self.generation_number = 0
        self.genes = []
        self.test_MSE = 0
        self.train_MSE = 0
        self.test_MAE = 0
        self.train_MAE = 0
        self.test_RMSE = 0
        self.train_RMSE = 0
        self.test_R2 = 0
        self.train_R2 = 0
        self.test_MAPE = 0
        self.train_MAPE = 0
        self.test_MAX_ERROR = 0
        self.train_MAX_ERROR = 0
        self.test_EXP_VAR_SCORE = 0
        self.train_EXP_VAR_SCORE = 0
        self.fitness = 0

    def __str__(self):
        return self.genes.__str__()

    def __repr__(self):
        return self.genes.__str__()

    def __eq__(self):
        return self.genes.__str__()

    def generate_genes(self, SEED):
        """
        Bu fonksiyon ile rassal bireyler üretilir.
        :param SEED: Benzer değerler üretmek için
        :return: Kromozoma gen eklenir.
        """
        np.random.seed(SEED + int(self.chr_id))  # sabitleme için gerekebilir.
        self.genes = []
        self.genes.append(np.around(
            np.random.uniform(learning_rate_ranges[0], learning_rate_ranges[1]),
            6))  # learning rate
        self.genes.append(np.around(
            np.random.uniform(dropout_rate_ranges[0], dropout_rate_ranges[1]),
            3))  # dropout rate
        self.genes.append(np.random.choice(hyper_parameters['layer_size']))  # layer size
        self.genes.append(np.random.choice(hyper_parameters['unit_size']))  # unit size
        self.genes.append(np.random.choice(hyper_parameters['activations']))  # activation function
        self.genes.append(np.random.choice(hyper_parameters['loss_functions']))  # loss function
        self.genes.append(list())
        if self.genes[2] == 1:
            self.genes[6].append(np.random.choice(hyper_parameters['models']))
        elif self.genes[2] == 2:
            self.genes[6].append(np.random.choice(hyper_parameters['models'], size=2).tolist())
        elif self.genes[2] == 4:
            self.genes[6].append(np.random.choice(hyper_parameters['models']))
        self.genes.append(np.random.choice(hyper_parameters['optimizers']))  # optimizer

    def change_genes(self, liste):
        """verilen l listesini chromosome genleri olarak alır"""
        self.genes = []
        self.genes = liste.copy()

    def calculate_fitness(self, notebook, epoch, batch_size):
        """
        Esas dosyayı çalıştıracak ve esas dosyadan dışarı aktaracak buradan alacak.
        :param notebook: Çalıştırılacak olan notebook dosyası
        :param epoch: Epoch sayısı
        :param batch_size: Batch sayısı
        :return:
        """
        print("Çalıştırılan genler", self.genes)
        model_number = str(self.generation_number) + '_' + str(self.chr_id)
        pm.execute_notebook('{}.ipynb'.format(notebook),
                            'notebooks/{}_{}.ipynb'.format(self.generation_number, self.chr_id),
                            parameters=dict(model_number=model_number, learning_rate=np.float64(self.genes[0]),
                                            dropout_rate=np.float64(self.genes[1]),
                                            layer_size=int(self.genes[2]),
                                            units=int(self.genes[3]),
                                            activation_function=self.genes[4],
                                            loss_function=self.genes[5], epoch=epoch, batch_size=batch_size,
                                            models=self.genes[6], optimizers=self.genes[7]))
        df_results = pd.read_csv(
            "performances/df_results{}.csv".format(model_number))
        self.test_MSE = df_results['MSE'][0]
        self.train_MSE = df_results['MSE'][1]
        self.test_MAE = df_results['MAE'][0]
        self.train_MAE = df_results['MAE'][1]
        self.test_RMSE = df_results['RMSE'][0]
        self.train_RMSE = df_results['RMSE'][1]
        self.test_R2 = df_results['R2'][0]
        self.train_R2 = df_results['R2'][1]
        self.test_MAPE = df_results['MAPE'][0]
        self.train_MAPE = df_results['MAPE'][1]
        self.test_MAX_ERROR = df_results['MAX_ERROR'][0]
        self.train_MAX_ERROR = df_results['MAX_ERROR'][1]
        self.test_EXP_VAR_SCORE = df_results['EXP_VAR_SCORE'][0]
        self.train_EXP_VAR_SCORE = df_results['EXP_VAR_SCORE'][1]
        self.fitness = min(self.test_MSE, 500)


class Population:
    def __init__(self, size, generation):
        self.generation = generation
        self.size = size
        self.chromosomes = []  # chromosome objeleri koyulacak.
        self.best, self.worst, self.avg = 0, 0, 0

    def init_pop(self):
        print("initializing population")
        for i in range(self.size):
            chrom = Chromosome(chr_id=i)
            chrom.generation_number = self.generation
            self.chromosomes.append(chrom)

    def sort_pop(self):
        return self.chromosomes.sort(key=lambda x: x.fitness, reverse=False)

    def calculate_best_avg_worst(self):
        self.sort_pop()
        self.best = round(self.chromosomes[0].fitness, 3)
        _total = 0
        for i, ind in enumerate(self.chromosomes):
            _total += ind.fitness
        self.avg = max(round(_total / self.size, 2), 500)
        self.worst = max(round(self.chromosomes[-1].fitness, 2), 500)

    def print_pop(self):
        ga_text_file = codecs.open("results\\ga-" + ".txt", "a+")
        ga_text_file.write(
            "--------------------------------------------------\n")
        ga_text_file.write(
            "Population #\t{}\tBest\t{}\tAvg\t{}\tWorst\t{}".format(self.generation, self.best, self.avg, self.worst))
        ga_text_file.write(
            "\n--------------------------------------------------\n")
        for i, x in enumerate(self.chromosomes):
            ga_text_file.write(
                "Individual #\t{}\t{}\tFitness\t{}\ttest_MSE\t{}\ttrain_MSE\t{}\ttest_MAE\t{}\ttrain_MAE\t{}\ttest_RMSE\t{}\ttrain_RMSE\t{}\ttest_R2\t{}\ttrain_R2\t{}\ttest_MAPE\t{}\ttrain_MAPE\t{}\ttest_MAX_ERROR\t{}\ttrain_MAX_ERROR\t{}\ttest_EXP_VAR_SCORE\t{}\ttrain_EXP_VAR_SCORE\t{}\n".format(
                    i, x.genes, round(x.fitness, 3), round(x.test_MSE, 3), round(x.train_MSE, 3), round(x.test_MAE, 3),
                    round(x.train_MAE, 3), round(x.test_RMSE, 3), round(x.train_RMSE, 3), round(x.test_R2, 3),
                    round(x.train_R2, 3), round(x.test_MAPE, 3), round(x.train_MAPE, 3), round(x.test_MAX_ERROR, 3),
                    round(x.train_MAX_ERROR, 3), round(x.test_EXP_VAR_SCORE, 3), round(x.train_EXP_VAR_SCORE, 3)))
        ga_text_file.close()

    def reap_pop(self):
        self.chromosomes = self.chromosomes[:self.size]

    def __str__(self):
        return self.chromosomes.__str__()

    def __repr__(self):
        return self.chromosomes.__str__()

In [ ]:
def crossover(p1, p2, points):
    if points <= 2:  # çaprazlamada 6. genin değişmemesi için önlem aldık.
        p1, p2 = list(p1), list(p2)
        c1, c2 = p1[:points] + p2[points:], p2[:points] + p1[points:]
    else:
        p1, p2 = list(p1), list(p2)
        c1, c2 = p1[:points] + p2[points:], p2[:points] + p1[points:]
        c1[6] = p1[6]
        c2[6] = p2[6]

    return c1, c2


def sum_to_x(n, x):
    values = [0.0, x] + list(np.random.uniform(low=0.0, high=x, size=n - 1))
    values.sort()
    return [values[i + 1] - values[i] for i in range(n)]

In [ ]:
SEED = 1234
nb_name = 'calistirma_genetic'
pop_list = []  # Popülasyon bireyleri

In [ ]:
#############################################################
# Initial Population
np.random.seed(SEED)

first_pop = Population(size=ga_parameters['population_size'], generation=0)
first_pop.init_pop()
for ind in first_pop.chromosomes:
    ind.generate_genes(SEED=SEED)
    ind.calculate_fitness(notebook=nb_name, epoch=hyper_parameters['epoch'],
                          batch_size=hyper_parameters['batch_size'])

first_pop.sort_pop()
#############################################################
# First Fitness Evaluation
first_pop.calculate_best_avg_worst()
first_pop.print_pop()
pop_list.append(first_pop)

In [ ]:
np.random.seed(SEED)
start_ga = time.time()
pop_list = pop_list[:1]
## Start iterations
for i in range(ga_parameters['max_num_iteration']):
    print("=" * 20, "Generation: {} started".format(i + 1), "=" * 20)
    crossover_points = np.random.choice(a=np.arange(1, len(hyper_parameters) - 1),
                                        size=ga_parameters['crossover_point_size'], replace=False, p=None)
    print("crossover_points: ", crossover_points)
    # 6. gen mutasyona uğramasın
    mutation_point_selection_list = [x for x in range(len(hyper_parameters)) if x != 6]
    mutation_points = np.random.choice(a=mutation_point_selection_list,
                                       size=ga_parameters['mutation_point_size'], replace=False, p=None)

    print("mutation_points: ", mutation_points)

    prev_pop = pop_list[i]
    new_pop = Population(size=ga_parameters['population_size'], generation=i + 1)
    new_pop.chromosomes = []
    chromosomes = prev_pop.chromosomes.copy()
    chr_list = np.array([chr.genes for chr in chromosomes], dtype=object)  # kromozomların genleri
    print("=" * 20, "Crossover started.", "=" * 20)
    crossover_idx = np.random.choice(range(ga_parameters['population_size']), replace=False,
                                     size=min(ga_parameters['crossover_chromosome_size'],
                                              ga_parameters['population_size']), p=ga_parameters['prob_chr'])
    crossover_pop = chr_list[crossover_idx, :]
    l = len(crossover_idx)
    crossover_pairs = np.array([x for x in zip(crossover_idx[:l // 2], crossover_idx[l // 2:])])
    for p1, p2 in crossover_pairs:
        parent1 = chr_list[p1, :]
        parent2 = chr_list[p2, :]
        random_number1 = np.random.random()
        print("random_number1: ", random_number1)
        if random_number1 <= ga_parameters['crossover_rate']:
            print("caprazlama yapılıyor")
            print(parent1, parent2, crossover_points)
            c1, c2 = crossover(parent1, parent2, crossover_points[0])
            child1 = Chromosome(chr_id=100 + i)
            child1.generation_number = i + 1
            child1.change_genes(c1)
            child2 = Chromosome(chr_id=200 + i)
            child2.generation_number = i + 1
            child2.change_genes(c2)
            child1.calculate_fitness(notebook=nb_name, epoch=hyper_parameters['epoch'],
                                     batch_size=hyper_parameters['batch_size'])
            child2.calculate_fitness(notebook=nb_name, epoch=hyper_parameters['epoch'],
                                     batch_size=hyper_parameters['batch_size'])
            new_pop.chromosomes.append(child1)
            new_pop.chromosomes.append(child2)
        else:
            print("caprazlama yapılmadı")
            parent1 = chromosomes[p1]
            parent1.generation_number = i + 1
            parent2 = chromosomes[p2]
            parent1.generation_number = i + 1
            new_pop.chromosomes.append(parent1)
            new_pop.chromosomes.append(parent2)

    print("=" * 20, "Crossover over.", "=" * 20)

    print("=" * 20, "Mutation started", "=" * 20)
    mutation_idx = np.random.choice(range(ga_parameters['population_size']), replace=False,
                                    size=min(ga_parameters['mutation_chromosome_size'],
                                             ga_parameters['population_size']), p=ga_parameters['prob_chr'])
    print(mutation_idx, ": mutation_idx", "mutation_idx type: ", type(mutation_idx))
    mutation_pop = [chromosomes[a] for a in mutation_idx]
    for mutation in mutation_pop:
        random_number2 = np.random.random()
        mutated_child = mutation.genes[:]
        if random_number2 <= ga_parameters['mutation_rate']:
            print("mutasyon yapılıyor")
            for point in mutation_points:
                if point == 0:  # learning rate
                    mutated_child[0] = np.around(np.random.uniform(learning_rate_ranges[0],
                                                                   learning_rate_ranges[1]), 6)
                elif point == 1:  # dropout rate
                    mutated_child[1] = np.around(np.random.uniform(dropout_rate_ranges[0],
                                                                   dropout_rate_ranges[1]), 3)
                elif point == 2:  # layer size
                    mutated_child[2] = np.random.choice(hyper_parameters['layer_size'])
                elif point == 3:  # unit size
                    mutated_child[3] = np.random.choice(hyper_parameters['unit_size'])
                elif point == 4:  # activation
                    mutated_child[4] = np.random.choice(hyper_parameters['activations'])
                elif point == 5:  # loss function
                    mutated_child[5] = np.random.choice(hyper_parameters['loss_functions'])
                elif point == 7:
                    mutated_child[7] = np.random.choice(hyper_parameters['optimizers'])

            if (mutated_child[2] == 1) or (mutated_child[2] == 4):
                mutated_child[6] = []
                mutated_child[6].append(np.random.choice(hyper_parameters['models']))
            elif mutated_child[2] == 2:
                mutated_child[6] = []
                mutated_child[6].append(np.random.choice(hyper_parameters['models'], size=2).tolist())

            mutated_chr = Chromosome(chr_id=300 + i)
            mutated_chr.generation_number = i + 1
            mutated_chr.change_genes(mutated_child)
            mutated_chr.calculate_fitness(notebook=nb_name, epoch=hyper_parameters['epoch'],
                                          batch_size=hyper_parameters['batch_size'])
            new_pop.chromosomes.append(mutated_chr)
        else:
            print("Mutasyon yapılmadı")
            mutation.generation_number = i + 1
            new_pop.chromosomes.append(mutation)

    print("=" * 20, "Mutation over.", "=" * 20)
    random_chromosome = Chromosome(chr_id=300 + i)
    random_chromosome.generation_number = i + 1
    random_genes = []
    random_genes.append(np.around(
        np.random.uniform(learning_rate_ranges[0], learning_rate_ranges[1]),
        6))  # learning rate
    random_genes.append(np.around(
        np.random.uniform(dropout_rate_ranges[0], dropout_rate_ranges[1]),
        3))  # dropout rate
    random_genes.append(np.random.choice(hyper_parameters['layer_size']))  # layer size
    random_genes.append(np.random.choice(hyper_parameters['unit_size']))  # unit size
    random_genes.append(np.random.choice(hyper_parameters['activations']))  # activation function
    random_genes.append(np.random.choice(hyper_parameters['loss_functions']))  # loss function
    random_genes.append(list())
    if random_genes[2] == 1:
        random_genes[6].append(np.random.choice(hyper_parameters['models']))
    elif random_genes[2] == 2:
        random_genes[6].append(np.random.choice(hyper_parameters['models'], size=2).tolist())
    elif random_genes[2] == 4:
        random_genes[6].append(np.random.choice(hyper_parameters['models']))
    random_genes.append(np.random.choice(hyper_parameters['optimizers']))  # optimizer

    random_chromosome.change_genes(random_genes)
    random_chromosome.calculate_fitness(notebook=nb_name, epoch=hyper_parameters['epoch'],
                                        batch_size=hyper_parameters['batch_size'])
    new_pop.chromosomes.append(random_chromosome)

    best_selection = chromosomes[0]
    best_selection.generation_number = i + 1
    new_pop.chromosomes.append(best_selection)  # en iyi kromozom
    new_pop.sort_pop()
    new_pop.chromosomes = new_pop.chromosomes[:ga_parameters['population_size']]
    #### Aynı genlere sahip bireylerin yerine rassal bireylerin oluşturulması
    seen_chrs = []
    new_chromosome_list = []
    for idx, obj in enumerate(new_pop.chromosomes):
        if obj.genes not in seen_chrs:
            print("obj.genes: ", obj.genes, "bulunmadı")
            seen_chrs.append(obj.genes)
        else:
            print("obj.genes: ", obj.genes, "bulundu")
            new_pop.chromosomes.pop(idx)

    while len(new_pop.chromosomes) < ga_parameters['population_size']:
        chr5 = Chromosome(chr_id=400 + i)

        random_genes2 = []
        random_genes2.append(np.around(
            np.random.uniform(learning_rate_ranges[0], learning_rate_ranges[1]),
            6))  # learning rate
        random_genes2.append(np.around(
            np.random.uniform(dropout_rate_ranges[0], dropout_rate_ranges[1]),
            3))  # dropout rate
        random_genes2.append(np.random.choice(hyper_parameters['layer_size']))  # layer size
        random_genes2.append(np.random.choice(hyper_parameters['unit_size']))  # unit size
        random_genes2.append(np.random.choice(hyper_parameters['activations']))  # activation function
        random_genes2.append(np.random.choice(hyper_parameters['loss_functions']))  # loss function
        random_genes2.append(list())
        if random_genes2[2] == 1:
            random_genes2[6].append(np.random.choice(hyper_parameters['models']))
        elif random_genes2[2] == 2:
            random_genes2[6].append(np.random.choice(hyper_parameters['models'], size=2).tolist())
        elif random_genes2[2] == 4:
            random_genes2[6].append(np.random.choice(hyper_parameters['models']))
        random_genes2.append(np.random.choice(hyper_parameters['optimizers']))  # optimizer

        chr5.change_genes(random_genes2)
        chr5.calculate_fitness(notebook=nb_name, epoch=hyper_parameters['epoch'],
                               batch_size=hyper_parameters['batch_size'])
        new_pop.chromosomes.append(chr5)

    new_pop.sort_pop()
    new_pop.calculate_best_avg_worst()
    new_pop.print_pop()
    pop_list.append(new_pop)

end_ga = time.time()
ga_cpu_text_file = codecs.open("results\\ga-CPU-" + ".txt", "a+")
ga_cpu_text_file.write(str(end_ga - start_ga))
ga_cpu_text_file.write("\n")
ga_cpu_text_file.close()
print("bitti..")

In [ ]:
# Populasyonu dışarı aktarma
import csv
with open('pop_list.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(
        ['generation', 'Individual', 'chromosome_ID', 'genes', 'fitness', 'test_EXP_VAR_SCORE', 'test_MAE', 'test_MAPE',
         'test_MAX_ERROR', 'test_MSE', 'test_R2', 'test_RMSE', 'train_EXP_VAR_SCORE', 'train_MAE', 'train_MAPE',
         'train_MAX_ERROR', 'train_MSE', 'train_R2', 'train_RMSE'])
    for gen_num, pop in enumerate(pop_list):
        for chr_rank, chr in enumerate(pop.chromosomes):
            writer.writerow(
                [gen_num, chr_rank, chr.chr_id, chr.genes, chr.fitness, chr.test_EXP_VAR_SCORE, chr.test_MAE,
                 chr.test_MAPE, chr.test_MAX_ERROR, chr.test_MSE, chr.test_R2, chr.test_RMSE, chr.train_EXP_VAR_SCORE,
                 chr.train_MAE, chr.train_MAPE, chr.train_MAX_ERROR, chr.train_MSE, chr.train_R2, chr.train_RMSE])